In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
train

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [2]:
train.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [3]:
import tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer()
# random_sentence="12th Fail: Did you know IPS officer Manoj Kumar Sharma, IRS officer Shraddha Joshi had cameos in Vikrant Massey film?"
def tokenizenews(random_sentence):
    tokenizer=tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts([random_sentence])
    sequence = tokenizer.texts_to_sequences([random_sentence])
    sequence_length = 50  # Adjust as needed
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    return padded_sequence
# train['article']=train['article'].apply(tokenizenews)
# train['highlights']=train['highlights'].apply(tokenizenews)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from tensorflow.keras.models import Model
def encoder(array):
    vocab_size = 10000
    embedding_dim = 200
    encoder_hidden_states=[]
    sequence_length = 50
    input_sequence = tf.keras.Input(shape=(sequence_length, ))
    embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
    encoder_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1600 , return_sequences=True))(embedding_layer)
    encoded_representation = tf.keras.layers.Concatenate()([encoder_lstm[:, -1, :], encoder_lstm[:, 0, :]])
    model = tf.keras.Model(inputs=input_sequence, outputs=encoded_representation )
    for random_sentence in array:
        result = model.predict(random_sentence)
        print(result.shape)
        encoder_hidden_states.append(result)
    return encoder_hidden_states

In [5]:
@tf.custom_gradient
def decoder(encoder_hidden_states,target_tokens,model1):
    loss = 0
    context_vector = None
    decoder_state = None
    sequence_length = 50
    hidden_size = 128
    embedding_dim = 128
    vocab_size = 10000
    context_vector=tf.zeros((128,50 ))
    decoder_state=tf.zeros((128, 50))
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    v = tf.Variable(tf.random.normal(shape=(hidden_size,50)), trainable=True, name='v')
    Wh = tf.Variable(tf.random.normal(shape=(hidden_size, hidden_size)), trainable=True, name='Wh')
    Ws = tf.Variable(tf.random.normal(shape=(hidden_size, embedding_dim)), trainable=True, name='Ws')
    battn = tf.Variable(tf.zeros(shape=(hidden_size,50)), trainable=True, name='battn')
    for t in range(len(target_tokens)):
        input_sequence = tf.keras.Input(shape=(sequence_length, ))
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
        model = tf.keras.Model(inputs=input_sequence, outputs=embedding_layer)
        output_embeddings = model.predict(target_tokens[t])
        attention_scores = []
        with tf.GradientTape() as tape:
            for i in range(len(encoder_hidden_states)):
                encoder_hidden_states[i] = tf.reshape(encoder_hidden_states[i], (128,50)) 
                e_t_i = v * tf.nn.tanh(tf.matmul(Wh , encoder_hidden_states[i])  + battn)
                attention_scores.append(e_t_i)
            attention_distributon=tf.nn.softmax(attention_scores)
            for i in range(len(encoder_hidden_states)):
                context_vector += attention_distributon[i] * encoder_hidden_states[i]
            output_embeddings=tf.reshape(output_embeddings, (128, 50, 1))
            output_embeddings=tf.squeeze(output_embeddings, axis=-1)
            concatenated_input = tf.expand_dims(tf.concat([context_vector,output_embeddings], axis=-1),axis=0)
#             input_sequence1 = tf.keras.Input(shape=(128, 100))
            concatenated_input=tf.stop_gradient(concatenated_input)
            output_embeddings1 = model1.predict(concatenated_input)
            vocab_distribution=tf.nn.softmax(output_embeddings1,axis=1)
            highest_prob_index=np.argmax(vocab_distribution)
            target_word_prob = vocab_distribution[0,highest_prob_index,0]
            timestep_loss = -tf.math.log(target_word_prob)
            decoder_state=tf.concat([decoder_state,context_vector], axis=-1)
            loss += timestep_loss
        all_variables = model1.trainable_variables + [v,Wh,Ws,battn]
        gradients=tape(loss/t,all_variables)
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        optimizer.apply_gradients(zip(gradients, all_variables))
        print(loss)

In [6]:
encoder_hidden=encoder([tokenizenews("Are you a curious mind, wanting to harness the latest developments in AI to transform the world? Or do you have the ability to invent for the future? EnCode is ready to provide you with a fantastic chance! UDGAM 2024, IIT Guwahati proudly presents the third edition of its flagship AI Hackathon: EnCode, presented by DevRev to inspire and support innovators!")])
summarize="EnCode, the flagship AI Hackathon presented by DevRev, is inviting curious minds and innovative individuals to participate in UDGAM 2024 at IIT Guwahati. This event offers a fantastic opportunity for those eager to harness the latest developments in AI and contribute to transforming the world. Whether you have a keen interest in AI or possess the ability to invent for the future, EnCode is ready to inspire and support you in this third edition of the hackathon."
target_tokens=[tokenizenews(summarize)]
input_sequence1 = tf.keras.Input(shape=(128, 100))
layer1=tf.keras.layers.Dense(128)(input_sequence1)
layer2=tf.keras.layers.Dense(1)(layer1)
model1 = tf.keras.Model(inputs=input_sequence1, outputs=layer2)
# decoder(encoder_hidden,target_tokens,model1)

1/1 [==============================] - 1s 981ms/step
(1, 6400)
